In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from scipy.stats import skew
import seaborn as sns
import sklearn
import warnings
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

/home/ryanryanadmin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_clean = pd.read_csv('train_clean.csv', dtype={'MSSubClass': str})  
train = pd.read_csv('train.csv', dtype={'MSSubClass': str})

In [3]:
coef_df = pd.read_csv('coef_df.csv')


Remove outlier records we previously identified as being harmful to the model

In [4]:
train_clean = train_clean[(train_clean.Id != 1299) & (train_clean.Id != 524)]

In [5]:
y_train = train_clean['SalePrice']
train_clean = train_clean.drop(['SalePrice', 'Id'], axis=1)

In [7]:
to_expand = list(coef_df.feature[:35].values)

In [8]:
train_no_expand = train_clean.drop(to_expand, axis=1)

In [9]:
train_to_expand = train_clean[to_expand]

In [10]:
poly = PolynomialFeatures(2)
train_to_expand = poly.fit_transform(train_to_expand)

In [11]:
X_train = np.hstack([train_to_expand, train_no_expand.values])

In [12]:
X_train.shape

(1458, 1051)

from sklearn.decomposition import PCA

#X_train = train_clean.drop(['SalePrice', 'Id'], axis=1)
pca = PCA()
pca.fit(to_expand)

ratios = np.cumsum(pca.explained_variance_ratio_)

ratios[ratios < 0.98].shape

pca = PCA(100)
X_train = pca.fit_transform(X_train)


feat_cols = coef_df.feature.values[:50]

## Fit the model.

Run grid search to fit find the best shrinkage hyperparameter.

In [16]:
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
def runGSAndGetRMSE(est, params):
    gs = GridSearchCV(est, param_grid= params, verbose=5, scoring='neg_mean_squared_error')
    gs.fit(X_train, y_train)
    print(np.sqrt(gs.best_score_*-1))
    print(gs.best_params_)
    return gs.best_estimator_

In [17]:
from sklearn.linear_model import Lasso
lassoParams = {'alpha': [0.0005,0.001, 0.005]}
bestLassoEst = runGSAndGetRMSE(Lasso(max_iter=10000), lassoParams)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] alpha=0.0005 ....................................................
[CV] .......................... alpha=0.0005, score=-0.013347 -   0.3s
[CV] alpha=0.0005 ....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] .......................... alpha=0.0005, score=-0.014743 -   0.3s
[CV] alpha=0.0005 ....................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


[CV] .......................... alpha=0.0005, score=-0.012117 -   0.3s
[CV] alpha=0.001 .....................................................
[CV] ........................... alpha=0.001, score=-0.012235 -   0.2s
[CV] alpha=0.001 .....................................................


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s


[CV] ........................... alpha=0.001, score=-0.015347 -   0.2s
[CV] alpha=0.001 .....................................................
[CV] ........................... alpha=0.001, score=-0.011708 -   0.1s
[CV] alpha=0.005 .....................................................
[CV] ........................... alpha=0.005, score=-0.013361 -   0.1s
[CV] alpha=0.005 .....................................................
[CV] ........................... alpha=0.005, score=-0.018744 -   0.1s
[CV] alpha=0.005 .....................................................
[CV] ........................... alpha=0.005, score=-0.013475 -   0.1s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    1.8s finished


0.114439810015
{'alpha': 0.001}


Fit our tuned model on all the data prior to submission.

In [72]:
bestLassoEst.fit(X_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Make predictions on the test set and undo our log transform so that the values will be on their original scale.

In [81]:
test = pd.read_csv('test_clean.csv', dtype={'MSSubClass': str})

In [82]:
ids = test.Id.values
test = test.drop('Id', axis=1)
test_to_expand = test[to_expand]
test_to_expand = poly.transform(test_to_expand)
test = test.drop(to_expand, axis=1)

test = np.hstack([test_to_expand, test.values])

In [83]:
preds = np.expm1(bestLassoEst.predict(test))
solution = pd.DataFrame({"id":ids, "SalePrice":preds}, columns=['id', 'SalePrice'])

solution.to_csv("lasso_expand_35.csv", index = False)

This one scores .12046

Let's save the model so we can use it later

In [115]:
from sklearn.externals import joblib
joblib.dump(bestLassoEst, 'poly_features_35_2.pkl')

['poly_features_35_2.pkl']